In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [22]:
df = pd.read_csv('../data/model_inputs/zips_with_clusters.csv')
#new_df = pd.read_csv('../data/processed/arrests_outside_buffer_by_zip_2016.csv')
other_df = pd.read_csv('../data/model_inputs/model_input.csv')

In [23]:
extra_columns = list(set(other_df.columns) - set(df.columns))
arrest_features = ['law_cat_cd', 'arrest_count']
dog_bit_features = ['dog_bit_count']
other_df

,year,zip,dog_bite_count,median_days_on_market,active_listing_count,median_listing_price,median_listing_price_per_square_foot,ADULT SERVICES,CAMPS,CHILD SERVICES AND WELFARE,...,TRANSPORTATION,VOCATIONAL AND PROPRIETARY SCHOOLS,WATER AND WASTEWATER,YOUTH SERVICES,cluster,arrest_year,law_cat_cd,arrest_count,revised_market_value,borough
0,2016,10001,4,86.5,71.0,3575000.0,2104.0,0.0,2.0,11.0,...,57.0,52.0,0.0,38.0,2.0,2016.0,F,6.0,4.866892e+06,Manhattan
1,2016,10001,4,86.5,71.0,3575000.0,2104.0,0.0,2.0,11.0,...,57.0,52.0,0.0,38.0,2.0,2016.0,F,6.0,4.866892e+06,Manhattan
2,2016,10001,4,86.5,71.0,3575000.0,2104.0,0.0,2.0,11.0,...,57.0,52.0,0.0,38.0,2.0,2016.0,M,3.0,4.866892e+06,Manhattan
3,2016,10001,4,86.5,71.0,3575000.0,2104.0,0.0,2.0,11.0,...,57.0,52.0,0.0,38.0,2.0,2016.0,M,3.0,4.866892e+06,Manhattan
4,2016,10002,18,62.0,107.0,1305000.0,1849.5,8.0,2.0,29.0,...,26.0,2.0,0.0,137.0,3.0,NaN,NaN,NaN,1.367783e+07,Manhattan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3567,2023,12414,1,54.0,25.0,488500.0,212.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3568,2023,12455,1,82.5,6.0,468500.0,292.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3569,2023,12543,1,34.0,3.5,301975.0,191.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3570,2023,12771,1,61.5,43.5,252250.0,171.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#!pip install geopandas
import geopandas as gpd

g_df = gpd.read_file('../data/raw/ny_new_york_zip_codes_geo.min.json')


def avg_val(zi):

    test = g_df[g_df['ZCTA5CE10'] == str(int(zi))]
    li = gpd.sjoin(g_df, test)['ZCTA5CE10_left'].astype(int).to_list()

    if int(zi) in li:
        li.remove(int(zi))
    if len(li) == 0:
        li.append(0)
    return df[df.zip.isin(li)].revised_market_value.mean()

In [5]:
df['neighbor_average'] = df.zip.map(avg_val)
df = df.dropna()

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['revised_market_value', 'Unnamed: 0', 'zip'], axis = 1), df[['revised_market_value']], test_size=0.2, random_state=42)

In [7]:
#!conda install python=3.11.0 --yes
!python --version

Python 3.10.15


In [8]:
#!pip install pycaret
#!pip install mlflow
from pycaret.regression import *

In [9]:
clf1 = setup(df.drop(['Unnamed: 0', 'zip'], axis = 1), target = 'revised_market_value', session_id=123, log_experiment=False, experiment_name='Cluster1')

,Description,Value
0,Session id,123
1,Target,revised_market_value
2,Target type,Regression
3,Original data shape,"(193, 28)"
4,Transformed data shape,"(193, 28)"
5,Transformed train set shape,"(135, 28)"
6,Transformed test set shape,"(58, 28)"
7,Numeric features,27
8,Preprocess,True
9,Imputation type,simple


In [10]:
best_model = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
huber,Huber Regressor,2382518.3965,46925929644131.2656,4478864.0993,-0.3467,1.4230,1.0520,0.0060
par,Passive Aggressive Regressor,3020349.9111,58183952872864.5859,5289376.0304,-1.0253,1.5775,1.4832,0.0040
dummy,Dummy Regressor,2923180.6000,43350338463334.3984,4656023.9750,-1.9149,1.5927,2.4365,0.0080
br,Bayesian Ridge,2923001.1625,43357415433830.3984,4656315.3625,-1.9149,1.5927,2.4366,0.0030
omp,Orthogonal Matching Pursuit,3041045.8125,45986077894246.3984,4976340.9000,-2.4668,1.6407,2.6663,0.0100
en,Elastic Net,3314048.0875,46155035390771.2031,5118739.1375,-4.4193,1.7262,2.9380,0.0060
ada,AdaBoost Regressor,2910748.5409,48768913631252.8906,5149722.7177,-5.2229,1.5511,2.0666,0.0090
knn,K Neighbors Regressor,3192897.2875,50483000246272.0000,5591025.8750,-6.0590,1.5417,2.4771,0.0080
rf,Random Forest Regressor,2992284.5202,48999152866529.6016,5483305.9509,-6.3519,1.5114,2.0620,0.0200
lightgbm,Light Gradient Boosting Machine,3280590.1953,47412067397216.5547,5351883.2125,-6.8043,1.7740,3.0518,0.0660


In [251]:
from sklearn.linear_model import HuberRegressor
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


reg = HuberRegressor()

pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('huber', reg)  
    ])


pipeline.fit(X_train, y_train)

comp = y_test.copy()
comp['pred'] = pipeline.predict(X_test)
comp['err'] = (abs(comp['revised_market_value']-comp['pred'])/comp['pred'])*100


m_comp = df[['Cluster', 'zip']].merge(comp, left_index = True, right_index = True)
m_comp.sort_values(by = 'err')

,Cluster,zip,revised_market_value,pred,err
134,1,11213.0,1.068843e+06,1.112107e+06,3.890283
154,1,11234.0,1.317275e+06,1.411530e+06,6.677497
15,2,10018.0,2.702798e+06,3.230768e+06,16.341937
164,1,11355.0,1.597785e+06,1.913453e+06,16.497319
18,2,10021.0,1.923310e+06,2.373688e+06,18.973753
133,0,11212.0,4.215769e+05,5.334795e+05,20.975982
174,3,11366.0,9.162027e+05,1.297529e+06,29.388669
83,0,10452.0,1.540250e+06,1.156438e+06,33.189099
90,1,10459.0,5.832229e+05,9.387598e+05,37.873048
92,1,10461.0,9.745397e+05,1.626643e+06,40.088907


In [252]:
i_df = pd.DataFrame(reg.coef_)
i_df.index = df.drop(['revised_market_value', 'Unnamed: 0', 'zip'], axis = 1).columns
i_df['p/n'] = '+'
i_df.loc[i_df[0] < 0, ['p/n']] = '-'
i_df[0] = abs(i_df[0])
i_df.sort_values(by = 0, ascending = False)

,0,p/n
HIGHER EDUCATION,189060.139805,+
HISTORICAL SITES,144558.313518,+
CHILD SERVICES AND WELFARE,141433.534825,-
ADULT SERVICES,131162.992062,+
VOCATIONAL AND PROPRIETARY SCHOOLS,125987.604300,+
CULTURAL INSTITUTIONS,114142.334869,+
PUBLIC SAFETY,96676.052599,-
JUSTICE AND CORRECTIONS,92808.769921,-
CAMPS,84823.707790,+
TRANSPORTATION,82339.186576,+
